In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import scipy.interpolate

In [2]:
# The Black Scholes Formula
# CallorPut
# S - Stock price (known)
# K - Strike price (known)
# T - Time to maturity (known)
# r - Riskfree interest rate (can be found)
# d - Dividend yield (KNown(?))
# v - Volatility (given)

def BlackScholes(CallorPut, S, K, T, r, d, v):
    d1 = (np.log(float(S)/K)+((r-d) + v*v/2.)*T)/(v*np.sqrt(T))
    d2 = d1 - v*np.sqrt(T)
    
    if CallorPut == 'Call':
        return S*np.exp(-d*T)*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*np.exp(-d*T)*norm.cdf(-d1)
    
##THIS WORKS DON'T TOUCH np

In [3]:
options = pd.read_csv('opt_port.csv')
options

,Maturity,Days,RF rate,Underlying,PutorCall,BoughtSold,Number of Shares,Strike,Vol,Div Yield,Spot
0,9/12/2019,129,0.010187,BHP,Call,Bought,20000,39.0,0.2600,0.0414,38.78
1,8/03/2020,219,0.009705,CBA,Put,Bought,15000,75.0,0.2798,0.0580,81.88
2,12/04/2020,254,0.009443,CBA,Call,Bought,15000,75.0,0.2798,0.0580,81.88
3,4/05/2020,276,0.009289,WESP,Put,Bought,25000,38.1,0.3056,0.0567,38.86
4,7/08/2020,371,0.009026,WESP,Call,Bought,20000,39.0,0.3298,0.0567,38.86
5,6/11/2020,462,0.008754,WPL,Call,Sold,10000,35.0,0.2845,0.0603,33.81
6,6/12/2020,492,0.008665,WPL,Put,Bought,15000,33.0,0.2700,0.0603,33.81


In [4]:
opt_list=(0,1,2,3,4,5,6)
options['BS_Price'] = ""
options

,Maturity,Days,RF rate,Underlying,PutorCall,BoughtSold,Number of Shares,Strike,Vol,Div Yield,Spot,BS_Price
0,9/12/2019,129,0.010187,BHP,Call,Bought,20000,39.0,0.2600,0.0414,38.78,
1,8/03/2020,219,0.009705,CBA,Put,Bought,15000,75.0,0.2798,0.0580,81.88,
2,12/04/2020,254,0.009443,CBA,Call,Bought,15000,75.0,0.2798,0.0580,81.88,
3,4/05/2020,276,0.009289,WESP,Put,Bought,25000,38.1,0.3056,0.0567,38.86,
4,7/08/2020,371,0.009026,WESP,Call,Bought,20000,39.0,0.3298,0.0567,38.86,
5,6/11/2020,462,0.008754,WPL,Call,Sold,10000,35.0,0.2845,0.0603,33.81,
6,6/12/2020,492,0.008665,WPL,Put,Bought,15000,33.0,0.2700,0.0603,33.81,


In [18]:
for x in opt_list:
    maturity = (options.iloc[x, 1]/365)
    options.iloc[x, 11] = BlackScholes( options.iloc[x, 4], #callorput
                 options.loc[x, 'Spot'], #spot
                 options.loc[x, 'Strike'], #Strike
                 maturity, # T 
                 options.iloc[x, 2], #RF rate
                 options.iloc[x, 9], #div yield
                 options.iloc[x, 8] #vol
                )
   

In [25]:
options['Price'] = options['BS_Price'] * options['Number of Shares']
options

,Maturity,Days,RF rate,Underlying,PutorCall,BoughtSold,Number of Shares,Strike,Vol,Div Yield,Spot,BS_Price,Delta,Price
0,9/12/2019,129,0.010187,BHP,Call,Bought,20000,39.0,0.2600,0.0414,38.78,2.06629,0.480676,41325.9
1,8/03/2020,219,0.009705,CBA,Put,Bought,15000,75.0,0.2798,0.0580,81.88,4.61319,-0.340071,69197.9
2,12/04/2020,254,0.009443,CBA,Call,Bought,15000,75.0,0.2798,0.0580,81.88,9.36883,0.610902,140532
3,4/05/2020,276,0.009289,WESP,Put,Bought,25000,38.1,0.3056,0.0567,38.86,4.27658,-0.451411,106914
4,7/08/2020,371,0.009026,WESP,Call,Bought,20000,39.0,0.3298,0.0567,38.86,4.05331,0.47565,81066.1
5,6/11/2020,462,0.008754,WPL,Call,Sold,10000,35.0,0.2845,0.0603,33.81,2.74806,0.407337,27480.6
6,6/12/2020,492,0.008665,WPL,Put,Bought,15000,33.0,0.2700,0.0603,33.81,4.73654,-0.456532,71048.1


In [12]:
# Black Scholes Call Delta 
# S - Stock price 
# K - Strike price 
# T - Time to maturity 
# r - Riskfree interest rate (can be found)
# d - Dividend yield 
# v - Volatility 
def bsdelta_call(S, K, T, r, d, v):
    if S>0 and K>0 and v!=0 and T > 0:
        
        term1 = np.log(S / K) + (r - d) * T
        term2 = v * v * T
        term3 = np.sqrt(term2)
        term4 = term2/2
        
        dPlus = (term1 + term4) / term3
        
        bsdelta_call = np.exp(-d * T) * norm.cdf(dPlus)
        
        return bsdelta_call
        
    else:
        
        bsdelta_call = 0
        return bsdelta_call
    
# Black Scholes put delta 
# S - Stock price 
# K - Strike price 
# T - Time to maturity 
# r - Riskfree interest rate (can be found)
# d - Dividend yield 
# v - Volatility 
def bsdelta_put(S, K, T, r, d, v):
    if S>0 and K>0 and v!=0 and T > 0:
        
        term1 = np.log(S / K) + (r - d) * T
        term2 = v * v * T
        term3 = np.sqrt(term2)
        term4 = term2/2
        
        dPlus = (term1 + term4) / term3
        
        bsdelta_put = -np.exp(-d * T) * norm.cdf(-dPlus)
        
        return bsdelta_put
        
    else:
        
        bsdelta_put = -np.exp(-d * T)
        return bsdelta_put
    
options['Delta'] = ""

for x in opt_list:
    maturity = (options.iloc[x, 1]/365)
    
    if options.iloc[x, 4] == 'Call':
        options.loc[x, 'Delta'] =  bsdelta_call(options.iloc[x, 10], #spot
                 options.iloc[x, 7], #Strike
                 maturity, # T 
                 options.iloc[x, 2], #RF rate
                 options.iloc[x, 9], #div yield
                 options.iloc[x, 8] #vol
                )
        #return bsdelta_call
        
    elif options.iloc[x, 4] == 'Put':
        options.loc[x, 'Delta'] = bsdelta_put(options.iloc[x, 10], #spot
                 options.iloc[x, 7], #Strike
                 maturity, # T 
                 options.iloc[x, 2], #RF rate
                 options.iloc[x, 9], #div yield
                 options.iloc[x, 8] #vol
                 )
        
        #return bsdelta_put
                     
    else:
        0

In [24]:
options

,Maturity,Days,RF rate,Underlying,PutorCall,BoughtSold,Number of Shares,Strike,Vol,Div Yield,Spot,BS_Price,Delta,Price
0,9/12/2019,129,0.010187,BHP,Call,Bought,20000,39.0,0.2600,0.0414,38.78,2.06629,0.480676,41325.9
1,8/03/2020,219,0.009705,CBA,Put,Bought,15000,75.0,0.2798,0.0580,81.88,4.61319,-0.340071,69197.9
2,12/04/2020,254,0.009443,CBA,Call,Bought,15000,75.0,0.2798,0.0580,81.88,9.36883,0.610902,140532
3,4/05/2020,276,0.009289,WESP,Put,Bought,25000,38.1,0.3056,0.0567,38.86,4.27658,-0.451411,106914
4,7/08/2020,371,0.009026,WESP,Call,Bought,20000,39.0,0.3298,0.0567,38.86,4.05331,0.47565,81066.1
5,6/11/2020,462,0.008754,WPL,Call,Sold,10000,35.0,0.2845,0.0603,33.81,2.74806,0.407337,27480.6
6,6/12/2020,492,0.008665,WPL,Put,Bought,15000,33.0,0.2700,0.0603,33.81,4.73654,-0.456532,71048.1


In [10]:
sharedata = pd.read_csv('RSKMShares.csv', index_col = 'Date', parse_dates = True)
sharedata.columns = [ 'ASX200', 'BHP', 'RIO', 'CBA', 'WESP', 'WPL', 'NAB', 'TLS']


In [26]:
X_vector = (#BHP
    
)options['Price'] * options['Delta']

0    19864.4
1   -23532.2
2    85851.6
3   -48262.4
4    38559.1
5    11193.9
6   -32435.7
dtype: object

In [27]:
X_vector = 19864.4 , (-23532.2 + 85851.6), (-48262.4 + 38559.1), (11193.9 - 32435.7)

In [28]:
sharelist = ('BHP', 'CBA', 'WESP', 'WPL')

share_log_returns = pd.DataFrame()

for x in sharelist:
    
    share_log_returns[x] = np.log(sharedata[x]).diff()
    
cov_matrix  = share_log_returns.cov()
cov_matrix

,BHP,CBA,WESP,WPL
BHP,0.000280,0.000088,0.000078,0.000147
CBA,0.000088,0.000149,0.000066,0.000077
WESP,0.000078,0.000066,0.000157,0.000067
WPL,0.000147,0.000077,0.000067,0.000231


In [29]:
Covar_X = cov_matrix.dot(X_vector)
X_Covar_X = Covar_X.dot(X_vector)
sqrt_covar = np.sqrt(X_Covar_X)
sqrt_covar

783.8084139234292

In [30]:
real_X =[566712.6640767439,
 284310.0,
 660154.7970677672,
 319729.44326595525,
 75240.5567134204,
 427650.0,
 119400.0]

In [ ]:
sharelist = ('BHP', 'RIO', 'CBA', 'WESP', 'WPL', 'NAB', 'TLS') 

Deltas_port = {
    'BHP' : ((options.loc[0, 'Delta'] * options.loc[0, 'Price']) 
             + shares.iloc[0,1]) * shares.loc[0, 'Spot'],
    'RIO' : ((shares.iloc[1,1]) * shares.loc[1, 'Spot']),
    'CBA' : ((options.loc[1, 'Delta'] * options.loc[1, 'Price']) 
             + (options.loc[2,'Delta'] * options.loc[2, 'Price'])
             + (shares.iloc[2,1] * shares.loc[2, 'Spot']) ),
    'WESP' : ((options.loc[3,'Delta'] * options.loc[3,'Price']) 
              + (options.loc[4,'Delta'] * options.loc[4,'Price']) 
              + (shares.iloc[3,1] * shares.loc[3, 'Spot'])), 
    'WPL' : ((options.loc[5,'Delta'] * options.loc[5,'Price']) 
             + (options.loc[6,'Delta'] * options.loc[6,'Price']) 
             + (shares.iloc[4,1]) * shares.loc[4, 'Spot']),
    'NAB' : (shares.iloc[5,1] * shares.loc[5, 'Spot']),
    'TLS' : (shares.iloc[6,1] * shares.loc[6, 'Spot'])
}